# Spacy Pipeline mit pandas

## Importe

In [2]:
import spacy
import pickle

import pandas as pd

## Laden der Daten

In [2]:
df = pd.read_xml('../data/Bundesregierung.xml')

In [3]:
# change dtype to datetime
df.loc[:, 'datum'] = pd.to_datetime(df.loc[:, 'datum'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2983 entries, 0 to 2982
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   person      2983 non-null   object        
 1   titel       2982 non-null   object        
 2   datum       2983 non-null   datetime64[ns]
 3   untertitel  2088 non-null   object        
 4   url         2983 non-null   object        
 5   anrede      1447 non-null   object        
 6   rohtext     2983 non-null   object        
 7   ort         690 non-null    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 186.6+ KB


## Doc-Objekt im pandas-Dataframe

Mit Hilfe einer Funktion kann für jede Zeile die Spalte 'rohtext' in ein Doc-Objekt umgewandelt  und in eine neue Spalte eingefügt werden.

Mit pickle kann der Dataframe gespeichert werden, sodass das Doc-Objekt erhalten bleibt. Allerdings ist dann die pickle-Datei mit fast 3.5 GB sehr groß. Wird der Dataframe wieder geladen, dann ist jedoch der Speicherbedarf des Arbeitsspeiche nicht sehr hoch.

### Funktion zum Erstellen des Doc-Objekts 

In [6]:
def create_doc_object(text, nlp):
    '''
    Loads SpaCy Language Model and creates a SpaCy Doc-Object.
    INPUT: string
    RETURN: spacy.tokens.doc.Doc
    '''       
        
    return nlp(text)    

In [5]:
nlp = spacy.load('de_core_news_md')

In [7]:
%%time

# Erstellen der Doc-Objekte
# kann ein paar Minuten laufen

df.loc[:, 'doc_object'] = df.loc[:, 'rohtext'].apply(lambda text: create_doc_object(text, nlp))

Wall time: 11min 27s


In [26]:
# speichern der Datei
# pickle-Datei ist fast 3.5 GB groß!

# df.to_pickle('../data/reden-bundesregierung.p')

### Einlesen der pickle-Datei

In [4]:
df_p = pd.read_pickle('../data/reden-bundesregierung.p')

In [20]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2983 entries, 0 to 2982
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   person      2983 non-null   object        
 1   titel       2982 non-null   object        
 2   datum       2983 non-null   datetime64[ns]
 3   untertitel  2088 non-null   object        
 4   url         2983 non-null   object        
 5   anrede      1447 non-null   object        
 6   rohtext     2983 non-null   object        
 7   ort         690 non-null    object        
 8   doc_object  2983 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 209.9+ KB


### Checks

In [21]:
type(df_p.loc[0, 'doc_object'])

spacy.tokens.doc.Doc

In [22]:
token_test = df_p.loc[0, 'doc_object']

In [29]:
for token in token_test[:10]:
    print(token.text, token.lemma_, token.pos_,token.ent_type_)

es ich PRON 
ist sein AUX 
angekündigt ankündigen VERB 
worden werden AUX 
, , PUNCT 
man man PRON 
sollte sollen AUX 
im im ADP 
Beruf Beruf NOUN 
zwischen zwischen ADP 


## Tokenisierung

In [143]:
def tokenize(doc):
    '''
    Tokenizes text using Doc-Object
    INPUT: Doc-Object
    RETURN: list with tokens
    '''
    return [ token.text for token in doc if not token.is_punct ]

In [144]:
%%time

df.loc[:, 'tokens'] = df.loc[:, 'doc_object'].apply(lambda doc: tokenize(doc))

Wall time: 2.57 s


In [145]:
df.loc[:, 'ntokens'] = df.loc[:, 'tokens'].apply(lambda tokens: len(tokens))

In [146]:
df.loc[:, 'ntokens'].describe()

count     2983.000000
mean      1939.533691
std       1285.096751
min          6.000000
25%        963.500000
50%       1599.000000
75%       2605.000000
max      10459.000000
Name: ntokens, dtype: float64

## Lemmatisierung

In [91]:
def lemmatize(doc):
    '''
    Lemmatizes text using Doc-Object
    INPUT: Doc-Object
    RETURN: list with lowercase lemmatas with stopwords and punctuation removed    
    '''
    # create stopwords taken from http://members.unine.ch/jacques.savoy/clef/germanST.txt
    with open('../data/stopwords.txt', 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    
    return [ token.lemma_.lower() for token in doc \
             if token.text not in stopwords \
             and token.is_alpha \
             and not token.is_punct ]

In [87]:
%%time

df.loc[:, 'lemmata'] = df.loc[:, 'doc_object'].apply(lambda doc: lemmatize(doc))

Wall time: 36.3 s


In [88]:
type(df.loc[0, 'lemmata'])

list

## NER

In [136]:
def extract_named_entities(doc, entity='PER'):
    '''
    Extracts named entities from Doc-Object.
    INPUT: Doc-Object
    RETURN: List with entities    
    '''
    return [ token.text for token in doc.ents if token.label_ == entity ]

In [137]:
%%time

entities = ['PER', 'ORG', 'LOC', 'MISC']

for entity in entities:
    df.loc[:, entity] = df.loc[:, 'doc_object'].apply(lambda doc: extract_named_entities(doc, entity=entity))

Wall time: 1.47 s


In [138]:
df.head(3).T

,0,1,2
person,Gerhard Schröder,Julian Nida-Rümelin,Gerhard Schröder
titel,Rede des Bundeskanzlers zur Eröffnung der 52. ...,Redebeitrag von Staatsminister Nida-Rümelin in...,Interview mit Bundeskanzler Schröder in 'Berli...
datum,2002-02-06 00:00:00,2002-02-01 00:00:00,2002-04-14 00:00:00
untertitel,"Man kann diese Filmfestspiele nicht eröffnen, ...","""Ich bin der Auffassung, wir müssen nicht nur ...",In dem Interview äußert sich Bundeskanzler Sch...
url,http://archiv.bundesregierung.de/bpaexport/red...,http://archiv.bundesregierung.de/bpaexport/red...,http://archiv.bundesregierung.de/bpaexport/red...
anrede,"Sehr geehrte Frau Nair, liebe Mitglieder der J...",Meine sehr geehrten Damen und Herren!,NaN
rohtext,"es ist angekündigt worden, man sollte im Beruf...","Frau Präsidentin! An Sie gerichtet, Herr Börn...",Frage (Peter Hahne): Bevor wir uns über den Au...
ort,None,None,None
doc_object,"(es, ist, angekündigt, worden, ,, man, sollte,...","(Frau, Präsidentin, !, , An, Sie, gerichtet, ...","(Frage, (, Peter, Hahne, ):, Bevor, wir, uns, ..."
tokens,"[es, ist, angekündigt, worden, man, sollte, im...","[Frau, Präsidentin, , An, Sie, gerichtet, Her...","[Frage, Peter, Hahne, Bevor, wir, uns, über, d..."


In [147]:
check_dtype = df.loc[0, 'tokens']

In [148]:
type(check_dtype)

list

In [149]:
type(check_dtype[0])

str

## POS

In [130]:
def extract_pos(doc, pos_tag='NOUN'):
    '''
    Extracts Part-of-Speech-Tag from doc-Object
    INPUT: Doc-Object
    RETURN: list with tokens
    '''
    return [ token.text.lower() for token in doc if token.pos_ == pos_tag ]

In [131]:
%%time

pos_tags = ['NOUN', 'VERB', 'ADJ']

for pos_tag in pos_tags:
    df.loc[:, pos_tag] = df.loc[:, 'doc_object'].apply(lambda doc: extract_pos(doc, pos_tag=pos_tag))

Wall time: 4.56 s


In [132]:
df.head(3).T

,0,1,2
person,Gerhard Schröder,Julian Nida-Rümelin,Gerhard Schröder
titel,Rede des Bundeskanzlers zur Eröffnung der 52. ...,Redebeitrag von Staatsminister Nida-Rümelin in...,Interview mit Bundeskanzler Schröder in 'Berli...
datum,2002-02-06 00:00:00,2002-02-01 00:00:00,2002-04-14 00:00:00
untertitel,"Man kann diese Filmfestspiele nicht eröffnen, ...","""Ich bin der Auffassung, wir müssen nicht nur ...",In dem Interview äußert sich Bundeskanzler Sch...
url,http://archiv.bundesregierung.de/bpaexport/red...,http://archiv.bundesregierung.de/bpaexport/red...,http://archiv.bundesregierung.de/bpaexport/red...
anrede,"Sehr geehrte Frau Nair, liebe Mitglieder der J...",Meine sehr geehrten Damen und Herren!,NaN
rohtext,"es ist angekündigt worden, man sollte im Beruf...","Frau Präsidentin! An Sie gerichtet, Herr Börn...",Frage (Peter Hahne): Bevor wir uns über den Au...
ort,None,None,None
doc_object,"(es, ist, angekündigt, worden, ,, man, sollte,...","(Frau, Präsidentin, !, , An, Sie, gerichtet, ...","(Frage, (, Peter, Hahne, ):, Bevor, wir, uns, ..."
tokens,"[es, ist, angekündigt, worden, man, sollte, im...","[Frau, Präsidentin, , An, Sie, gerichtet, Her...","[Frage, Peter, Hahne, Bevor, wir, uns, über, d..."


## Speichern der Resultate

### als csv

In [150]:
df.to_csv('../data/reden-bundesregierung-preprocessed-with-doc-object.csv', index=False)

In [151]:
df_ohne_doc = df.drop(['doc_object'], axis=1)

In [152]:
df_ohne_doc.to_csv('../data/reden-bundesregierung-preprocessed.csv', index=False)

### als pickle

In [153]:
df_ohne_doc.to_pickle('../data/reden-bundesregierung-preprocessed.p')

## checks

### from csv

In [154]:
df_1 = pd.read_csv('../data/reden-bundesregierung-preprocessed-with-doc-object.csv')

In [155]:
df_1.head(3).T

,0,1,2
person,Gerhard Schröder,Julian Nida-Rümelin,Gerhard Schröder
titel,Rede des Bundeskanzlers zur Eröffnung der 52. ...,Redebeitrag von Staatsminister Nida-Rümelin in...,Interview mit Bundeskanzler Schröder in 'Berli...
datum,2002-02-06,2002-02-01,2002-04-14
untertitel,"Man kann diese Filmfestspiele nicht eröffnen, ...","""Ich bin der Auffassung, wir müssen nicht nur ...",In dem Interview äußert sich Bundeskanzler Sch...
url,http://archiv.bundesregierung.de/bpaexport/red...,http://archiv.bundesregierung.de/bpaexport/red...,http://archiv.bundesregierung.de/bpaexport/red...
anrede,"Sehr geehrte Frau Nair, liebe Mitglieder der J...",Meine sehr geehrten Damen und Herren!,NaN
rohtext,"es ist angekündigt worden, man sollte im Beruf...","Frau Präsidentin! An Sie gerichtet, Herr Börn...",Frage (Peter Hahne): Bevor wir uns über den Au...
ort,NaN,NaN,NaN
doc_object,"es ist angekündigt worden, man sollte im Beruf...","Frau Präsidentin! An Sie gerichtet, Herr Börn...",Frage (Peter Hahne): Bevor wir uns über den Au...
tokens,"['es', 'ist', 'angekündigt', 'worden', 'man', ...","['Frau', 'Präsidentin', ' ', 'An', 'Sie', 'ger...","['Frage', 'Peter', 'Hahne', 'Bevor', 'wir', 'u..."


In [158]:
type(df_1.loc[0, 'doc_object'])

str

In [159]:
type(df_1.loc[0, 'tokens'])

str

In [164]:
df_1b = pd.read_csv('../data/reden-bundesregierung-preprocessed.csv')

In [165]:
type(df_1.loc[0, 'tokens'])

str

In [174]:
ORG_Test = df_1b.loc[0, 'LOC']

In [175]:
ORG_Test[0]

'['

In [176]:
type(ORG_Test)

str

In [177]:
type(ORG_Test[0])

str

### from pickle

In [160]:
df_2 = pd.read_pickle('../data/reden-bundesregierung-preprocessed.p')

In [162]:
type(df_2.loc[0, 'tokens'])

list

In [163]:
df_2.head(3).T

,0,1,2
person,Gerhard Schröder,Julian Nida-Rümelin,Gerhard Schröder
titel,Rede des Bundeskanzlers zur Eröffnung der 52. ...,Redebeitrag von Staatsminister Nida-Rümelin in...,Interview mit Bundeskanzler Schröder in 'Berli...
datum,2002-02-06 00:00:00,2002-02-01 00:00:00,2002-04-14 00:00:00
untertitel,"Man kann diese Filmfestspiele nicht eröffnen, ...","""Ich bin der Auffassung, wir müssen nicht nur ...",In dem Interview äußert sich Bundeskanzler Sch...
url,http://archiv.bundesregierung.de/bpaexport/red...,http://archiv.bundesregierung.de/bpaexport/red...,http://archiv.bundesregierung.de/bpaexport/red...
anrede,"Sehr geehrte Frau Nair, liebe Mitglieder der J...",Meine sehr geehrten Damen und Herren!,NaN
rohtext,"es ist angekündigt worden, man sollte im Beruf...","Frau Präsidentin! An Sie gerichtet, Herr Börn...",Frage (Peter Hahne): Bevor wir uns über den Au...
ort,None,None,None
tokens,"[es, ist, angekündigt, worden, man, sollte, im...","[Frau, Präsidentin, , An, Sie, gerichtet, Her...","[Frage, Peter, Hahne, Bevor, wir, uns, über, d..."
ntokens,1911,1319,1512


In [170]:
ORG_Test = df_2.loc[0, 'LOC']

In [171]:
ORG_Test[0]

'Amerika'

In [172]:
type(ORG_Test)

list

In [173]:
type(ORG_Test[0])

str